## Kream Products 크롤러
- products 정보를 크롤링합니다.
- 제품 목록
    - nike dunk low
    - nike dunk high
    - nike air force 1
    - nike air max 97
    - jordan 1 low
    - jordan 1 high
    - adidas yeezy boost 350 v2
    - adidas yeezy boost 700
    - adidas yeezy 500
    - new balance 992
    - new balance 327
    - new balance 2002

In [1]:
import time
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.keys import Keys

import pandas as pd
import numpy as np

pd.options.display.max_rows=100
pd.options.display.max_columns=100

### 제품 리스트 생성

In [2]:
product = 'nike dunk'
brand = 'Nike'
category = 'Nike Dunk'
# collection = '60'

### 크롤링 환경 설정

In [3]:
# 크롬창 열기
driver = webdriver.Chrome('./chromedriver')

# 윈도우 사용자분들
# driver = webdriver.Chrome('.//chromedriver.exe')

<ipython-input-3-8ffe0452779f>:2: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome('./chromedriver')


In [18]:
# product 검색결과 크림사이트 접속
# url = 'https://kream.co.kr/search?category_id=1&tag_id[collection]={0}&sort=popular&per_page=40'.format(collection)
url = 'https://kream.co.kr/search?category_id=1&keyword={0}&sort=popular&per_page=40'.format(product)
driver.get(url)

In [4]:
import datetime
    
def doScrollDown(whileSeconds):
    start = datetime.datetime.now()
    end = start + datetime.timedelta(seconds=whileSeconds)
    while True:
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight)")
        time.sleep(1)
        if datetime.datetime.now() > end:
            break

In [5]:
doScrollDown(10)

In [6]:
html = driver.page_source
soup = BeautifulSoup(html, 'html.parser')

In [11]:
product_k_list = []
for item in soup.select('p.translated_name'):
    product_k_list.append(item.text)

In [12]:
number_list = []
for item in soup.select('div.search_result_item > a.item_inner'):
    number_list.append(item)
product_list = []
for item in soup.select('div.title > p.name'):
    product_list.append(item.text)
wish_list = []
for item in soup.select('span.wish_figure'):
    wish_list.append(item.text.replace(' ',''))
review_list = []
for item in soup.select('span.review_figure'):
    review_list.append(item.text.replace(' ',''))

In [13]:
def number_extract(x):
    r1 = str(x).find('products/')
    r2 = str(x)[r1+9:].find('"')
    return str(x)[r1+9:r1+9+r2]

In [14]:
number_list = [number_extract(item) for item in number_list]

In [15]:
df = pd.DataFrame({'product_k':product_k_list, 'product':product_list, 'brand':brand, 'category':category,  'number':number_list, 'wish':wish_list, 'review':review_list})

In [16]:
df

,product_k,product,brand,category,number,wish,review
0,나이키 덩크 로우 레트로 블랙,Nike Dunk Low Retro Black,Nike,Nike Dunk,28029,5.3만,"7,883"
1,(W) 나이키 덩크 로우 블랙,(W) Nike Dunk Low Black,Nike,Nike Dunk,28030,3.9만,"2,969"
2,(W) 나이키 덩크 로우 베니스,(W) Nike Dunk Low Venice,Nike,Nike Dunk,47877,"3,057",33
3,나이키 덩크 로우 프리미엄 시엠프레 파밀리아,Nike Dunk Low Premium Siempre Familia,Nike,Nike Dunk,47119,"1,025",26
4,(GS) 나이키 덩크 로우 블랙,(GS) Nike Dunk Low Black,Nike,Nike Dunk,28229,1.7만,"1,063"
...,...,...,...,...,...,...,...
355,(W) 나이키 덩크 로우 디스럽트 파이어크래커,(W) Nike Dunk Low Disrupt Firecracker,Nike,Nike Dunk,29013,170,4
356,(PS) 나이키 덩크 하이 레트로 바스트 그레이,(PS) Nike Dunk High Retro Vast Grey,Nike,Nike Dunk,29654,476,8
357,(PS) 나이키 x 션 클리버 SB 덩크 로우 홀리데이 스페셜,(PS) Nike x Sean Cliver SB Dunk Low Holiday Sp...,Nike,Nike Dunk,27472,166,6
358,(PS) 나이키 x 메디콤 토이 SB 덩크 로우 베어브릭,(PS) Nike x Medicom Toy SB Dunk Low BE@RBRICK,Nike,Nike Dunk,25778,11,0


In [17]:
df.to_csv('data/products/products_{0}2.csv'.format(product.replace(' ','_')))